# Take Home Project: Wrangling coal mine data

### 1. Do the work!
The task is to **write code to extract and clean** some data about coal mines, and **write notes on how to integrate that work into an existing system**.

**Setup instructions are in the repo `README.md`.**

* This task is an example of the kind of work we do to make public energy data usable for analysis. We want to get at a few different angles:
  * How do you approach building code from scratch?
  * How do you think about messy datasets?
  * How do you think about testing software that deals with said messy datasets?
  * How do you think about integrating code into an existing system?
* **Spend up to two hours working on this.** If you don't finish, don't worry! **Do** try to make sure you have a little bit of both sections (coding and writing) done, though. The point is to have something concrete and technical to talk about at all.
* Feel free to use whatever documentation or online resources you would normally consult while working on a data wrangling problem.
* Feel free to use additional 3rd party libraries if you want to.  You should be able to install them from within the notebook using `!pip install packagename` or `!conda install packagename`

### 2. Email us your notebook!
* Send it to [hello@catalyst.coop](mailto:hello@catalyst.coop).
* We'll review your notebook and if it looks good, we'll reach out to schedule a longer conversation about it.

### 3. Follow-up conversation

If we think your notebook looks good, we'll schedule a 60 minute conversation about it.

You will need this code running on your computer and we will expect you to share your screen. Some of the questions will also be best answered by quickly writing out some pseudo-code.

During this conversation, we'll ask you to walk us through your Python code for the extraction piece and your English words for the integration piece.

We'll also ask some of the following questions:

* What assumptions are you making about the input data?
  * How will you test whether / when those assumptions are valid?
  * How would you / did you deal with the data that don’t conform to those assumptions?
* What expectations do you have about the output data?
  * How will you evaluate the completeness of the data that you’ve been able to extract?
  * What kind of queries are you trying to make easy with the structure of the output data?
  * What kind of data validation checks would you design to make sure that the output meets your expectations? These could be either integrated into the table transformation process, or run on the final output.
* Did you try anything that didn't work? What was it?
* If there are records which can’t be reasonably cleaned automatically, but were high value in an advocacy context, how would you integrate manual cleaning into the automated process so that the manual effort is captured, and can be incrementally improved over time?
* How do you decide when data isn’t recoverable?
* What parts of this process might make sense to generalize / abstract for re-use in extracting, cleaning, and reorganizing data from other tables?

## Background on the MSHA Coal Mine Data

* The Mine Health and Safety Administration (MSHA) collects a variety of information about mines, incuding who owns them, what and how much they produce, mining methods used, environmental and safety violations, number of employees, ownership, and location.
* This information can be helpful for understanding the economic and environmental consequences of shutting down coal fired power plants. It's especially relevant right now, since the Inflation Reduction Act (IRA) provides tax benefits for clean energy projects in former coal communities. (You can read more about "energy and coal communitites" [here](https://www.resources.org/common-resources/what-is-an-energy-community/), but that's not required to answer this interview question.)

## Extract and Clean the MSHA Mine Data Set

Please write code in this notebook to address the following requirements.

### Extract

* Design and implement a function or class that can be used to extract the [MSHA Mines Data Set (ZIP)](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines.zip).
    * The input to this function can be a URL or local path to the published zipfile. You don't have to worry about handling both.
    * The output should be a `pandas` dataframe.


### Transform/Clean
Take the extracted MSHA Mines data frame and impose some order on it, in preparation for loading it into a well-normalized relational database.

* There are many columns - clean 5-10 that seem the most important to you. We'd like to be able to answer questions like:
  * Which companies control/operate the most mines?
  * How many people are employed directly at coal mines in your state?
  * How many mines have been built but aren't yet operational?
* Any columns you've cleaned should end up with well-defined data types.
* Write down any properties you'd like to assert about the input or output data. If you have time left after everything, we'd love it if you could automate some of these assertions.
* Write down any questions you still have about the data by the end of your time window.


### Some hints
* The MSHA dataset has a [Definitions File](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines_Definition_File.txt) with column type and description information. 
* You'll need to use the `latin_1` character encoding when extracting the .txt files.

## Preliminary checking and set up
Examining the [MSHA Data File](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines_Definition_File.txt), these are the columns that I believe should be extracted and why
1. **MINE_ID**: ID specifically designated by the MSHA, could be used as a index/key incase future users want to cross-reference this database with one that just has the Mine ID and no MINE name. 
2. **CURRENT_MINE_STATUS**: The values, "Abandoned, Abandoned and Sealed, Active, Intermittent, New Mine, NonProducing and Temporarily Idled." lets me know if the existing mine is operating or not. 
    * Side Question: How to know which mine is specifically new and not operational. Maybe referencing the NonProducing and Temporarily Idled status to the last date this was assigned and the date the mine was opened/logged? 
3. **CURRENT_STATUS_DT**: Let's me know when the status was given, helpful to cross-reference for new and not yet operational mines instead of just ALL non-operational miles.  
4. **CURRENT_CONTROLLER_BEGIN_DT**: Just backup, to know which mines are "new" and which are "old"
    * How does one determine when a mine is "old"? Important because it helps with the extracting of mines that are after a certain date. But if function is to be constantly used, how to write code that is relational to present time and not just time the code was written? 
5. **CURRENT_CONTROLLER_NAME**: Lets us know what entity owns the mine, useful for extracting in question 3. 
6. **CURRENT_OPERATOR_ID**: Useful ID incase people using this need to run this against another database that just has Opertator IDs and not names. 
7. **STATE**: Gives me the state of the mines. Use this to then derive just NY. 
8. **COMPANY_TYPE**: Helps disaggregate what type of companies hold the most mines. 
9. **NO_EMPLOYEES**: Useful for question 2. 

## 1. Figure out how I would manually import and clean this data
This is important because these are going to be the parts of my function, the parts make the machine. I also need to manually do it first because I want to see how states are logged. (Knowing that, I can extract only NY) 
1. The parts: I think what I am going to need are: open, read, and read_csv
2. After that, using the prompt, I figure out how to extract the columns i need. 
3. REMEMBER: Ending result MUST be a pandas dataframe

In [2]:
## import necessary libraries
import pandas as pd
import io

In [3]:
##bringing in the file 
with open("Mines.txt", "r", encoding="latin-1") as mines:
    rawMines = mines.read()

In [4]:
## what IS rawMines? so I know what to run to turn it into the preliminary rawDF. 
## From the rawDF, I'll start extracting the columns i need. 
type(rawMines)

str

In [5]:
raw_df = pd.read_csv(io.StringIO(rawMines), sep="|")
raw_df

,MINE_ID,CURRENT_MINE_NAME,COAL_METAL_IND,CURRENT_MINE_TYPE,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_ID,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,CURRENT_OPERATOR_NAME,...,NO_NONPRODUCING_PITS,NO_TAILING_PONDS,PILLAR_RECOVERY_USED,HIGHWALL_MINER_USED,MULTIPLE_PITS,MINERS_REP_IND,SAFETY_COMMITTEE_IND,MILES_FROM_OFFICE,DIRECTIONS_TO_MINE,NEAREST_TOWN
0,100003,O'Neal Quarry & Mill,M,Surface,Active,01/22/1979,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",...,NaN,0.0,N,N,N,N,N,100,Approx 7 miles south of the city of Alabaster ...,Calera
1,100004,Brierfield Quarry,M,Surface,Active,03/04/2003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",...,NaN,0.0,N,N,N,N,N,35,"I-65 SOUTH EXIT US 31 TO CALERA, TURN RIGHT ON...",Brierfield
2,100005,Birmingham Plant,M,Surface,Abandoned,08/15/1989,0041044,Lhoist Group,L10998,Allied Lime Company,...,NaN,NaN,N,N,N,N,N,0,NaN,NaN
3,100006,Auburn Quarry,M,Surface,Active,09/24/1976,M00174,Martin Marietta Materials Inc,L16197,"Martin Marietta Materials, Inc.",...,NaN,0.0,N,N,N,N,N,140,85 to Hwy. 29 South to County Rd. 10; turn lef...,Auburn
4,100008,Landmark Plant,M,Surface,Active,11/14/1975,M31753,Alan B Cheney,L31753,Cheney Lime & Cement Company,...,NaN,0.0,N,N,N,N,N,25,I-65 S to exit 238; right on Hwy 31 1/8 miles ...,Alabaster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90941,6900009,CROSS ISLAND ROAD,M,Surface,New Mine,10/24/2016,0125183,Hang Cheing Tan,0145154,"GPPC, INC",...,NaN,NaN,N,N,N,N,N,0,Coming from Kagman goint to Capitol Hill right...,Capital Hill
90942,6900010,United Construction Corporation,M,Surface,New Mine,11/09/2016,0133643,Xiaojun Ying,0151233,United Construction Corporation,...,NaN,NaN,N,N,N,N,N,0,"From the airport to Dandan, to San Vincente, g...",chalan laulau
90943,6900011,New Hope Quarry,M,Surface,New Mine,08/15/2019,NaN,NaN,NaN,SUCCESS INTERNATIONAL CORPORATION,...,NaN,NaN,N,N,N,N,N,0,UNKNOWN,NONE/ABANDONED
90944,6900012,Sinapalo Quarry 1 &2,M,Surface,New Mine,09/02/2021,NaN,NaN,NaN,General Pacific Services,...,NaN,NaN,N,N,N,N,N,0,UNKNOWN,Sinapalo


In [6]:
##extracting the columns I need.
allmines_df = raw_df[["MINE_ID","CURRENT_MINE_STATUS", "CURRENT_STATUS_DT", "CURRENT_CONTROLLER_BEGIN_DT",
              "CURRENT_CONTROLLER_NAME", "CURRENT_OPERATOR_ID", "STATE", "COMPANY_TYPE", "NO_EMPLOYEES"]]
allmines_df

,MINE_ID,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_BEGIN_DT,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,STATE,COMPANY_TYPE,NO_EMPLOYEES
0,100003,Active,01/22/1979,07/01/1989,Lhoist Group,L13586,AL,Other,108.0
1,100004,Active,03/04/2003,06/14/2000,Lhoist Group,L13586,AL,Other,28.0
2,100005,Abandoned,08/15/1989,07/31/1989,Lhoist Group,L10998,AL,Other,NaN
3,100006,Active,09/24/1976,01/17/2002,Martin Marietta Materials Inc,L16197,AL,Corporation,15.0
4,100008,Active,11/14/1975,01/01/1950,Alan B Cheney,L31753,AL,Corporation,33.0
...,...,...,...,...,...,...,...,...,...
90941,6900009,New Mine,10/24/2016,10/24/2016,Hang Cheing Tan,0145154,MP,Corporation,NaN
90942,6900010,New Mine,11/09/2016,11/09/2016,Xiaojun Ying,0151233,MP,Corporation,NaN
90943,6900011,New Mine,08/15/2019,NaN,NaN,NaN,MP,NaN,NaN
90944,6900012,New Mine,09/02/2021,NaN,NaN,NaN,MP,NaN,NaN


In [39]:
##specifying the state I need, which is NY 
NYS_mines = allmines_df.query('STATE == "NY"')
NYS_mines

,MINE_ID,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_BEGIN_DT,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,STATE,COMPANY_TYPE,NO_EMPLOYEES
47288,3000002,Abandoned,08/01/1982,01/01/1950,Alpha Portland Industries Inc,L00108,NY,Other,NaN
47289,3000003,Intermittent,08/30/2023,08/28/2023,Josh Axtell,0177773,NY,Other,1.0
47290,3000006,Active,02/23/1983,01/01/2019,Holcim Ltd,0051046,NY,Corporation,189.0
47291,3000007,Active,06/24/1985,10/18/2010,David George Kasmoch,0115065,NY,Corporation,22.0
47292,3000009,Active,06/07/2023,12/14/1979,Colas S A,L03974,NY,Corporation,11.0
...,...,...,...,...,...,...,...,...,...
49810,3004143,Intermittent,09/20/2024,NaN,NaN,NaN,NY,NaN,4.0
49811,3004144,Intermittent,11/07/2024,10/03/2024,Warren W Fane,L11719,NY,Corporation,2.0
49812,3004145,New Mine,11/18/2024,NaN,NaN,NaN,NY,NaN,NaN
62561,3609813,Abandoned,05/03/2010,11/23/2009,Jonathan Schilowitz,L00045,NY,Corporation,0.0


## 2. Now to automate this
Now that i have done this manually, I have the parts. I just have to now take the parts and make an automated function.  
The parts: 
   * opening txt
   * turn string to df
   * extract specific columns
   * query just for NY 
   * return the df

In [58]:
##writing the function

def NYmines(file):
    '''
    Takes the data file you feed it and turns it into a dataframe that only includes mines in NY state 
    '''
    with open(file, "r", encoding="latin-1") as mines:
        rawMines = mines.read()
    print("grabbed file, creating raw df") ##updates the user and lets us know which part of the function breaks
    
    
    raw_df = pd.read_csv(io.StringIO(rawMines), sep="|")
    raw_df
    print("extracting needed columns")
    
    allmines_df = raw_df[["MINE_ID","CURRENT_MINE_STATUS", "CURRENT_STATUS_DT", "CURRENT_CONTROLLER_BEGIN_DT",
              "CURRENT_CONTROLLER_NAME", "CURRENT_OPERATOR_ID", "STATE", "COMPANY_TYPE", "NO_EMPLOYEES"]]
    print("grabbing only NY mines ")
    
    ny_mines = allmines_df.query('STATE == "NY"')
    print(type((ny_mines))) ##test to see if the returning piece IS a dataframe
    print("your NY mines are stored in the df called ny_mines")
    return ny_mines

In [54]:
##testing the function

def NYmines("Mines.txt")

SyntaxError: invalid syntax (555168400.py, line 3)

In [55]:
##giving it the file/path name instead because opening zip files becoming a little tricky and time constraint
rawData = "/Users/sajinashrestha/Documents/dataProjects/test/Mines.txt"

In [59]:
NYmines(rawData)

grabbed file, creating raw df
extracting needed columns
grabbing only NY mines 
<class 'pandas.core.frame.DataFrame'>
your NY mines are stored in the df called ny_mines


,MINE_ID,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_BEGIN_DT,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,STATE,COMPANY_TYPE,NO_EMPLOYEES
47288,3000002,Abandoned,08/01/1982,01/01/1950,Alpha Portland Industries Inc,L00108,NY,Other,NaN
47289,3000003,Intermittent,08/30/2023,08/28/2023,Josh Axtell,0177773,NY,Other,1.0
47290,3000006,Active,02/23/1983,01/01/2019,Holcim Ltd,0051046,NY,Corporation,189.0
47291,3000007,Active,06/24/1985,10/18/2010,David George Kasmoch,0115065,NY,Corporation,22.0
47292,3000009,Active,06/07/2023,12/14/1979,Colas S A,L03974,NY,Corporation,11.0
...,...,...,...,...,...,...,...,...,...
49810,3004143,Intermittent,09/20/2024,NaN,NaN,NaN,NY,NaN,4.0
49811,3004144,Intermittent,11/07/2024,10/03/2024,Warren W Fane,L11719,NY,Corporation,2.0
49812,3004145,New Mine,11/18/2024,NaN,NaN,NaN,NY,NaN,NaN
62561,3609813,Abandoned,05/03/2010,11/23/2009,Jonathan Schilowitz,L00045,NY,Corporation,0.0


In [57]:
##testing to see if the returned thing is actually a dataframe
type(ny_mines)

NameError: name 'ny_mines' is not defined

In [61]:
##this not defined makes me nervous because it's not stored outside the function. for actual use, I would do:
ny_mines_df = NYmines(rawData)

grabbed file, creating raw df
extracting needed columns
grabbing only NY mines 
<class 'pandas.core.frame.DataFrame'>
your NY mines are stored in the df called ny_mines


In [63]:
##now it is stored in a df that we can actually use
ny_mines_df

,MINE_ID,CURRENT_MINE_STATUS,CURRENT_STATUS_DT,CURRENT_CONTROLLER_BEGIN_DT,CURRENT_CONTROLLER_NAME,CURRENT_OPERATOR_ID,STATE,COMPANY_TYPE,NO_EMPLOYEES
47288,3000002,Abandoned,08/01/1982,01/01/1950,Alpha Portland Industries Inc,L00108,NY,Other,NaN
47289,3000003,Intermittent,08/30/2023,08/28/2023,Josh Axtell,0177773,NY,Other,1.0
47290,3000006,Active,02/23/1983,01/01/2019,Holcim Ltd,0051046,NY,Corporation,189.0
47291,3000007,Active,06/24/1985,10/18/2010,David George Kasmoch,0115065,NY,Corporation,22.0
47292,3000009,Active,06/07/2023,12/14/1979,Colas S A,L03974,NY,Corporation,11.0
...,...,...,...,...,...,...,...,...,...
49810,3004143,Intermittent,09/20/2024,NaN,NaN,NaN,NY,NaN,4.0
49811,3004144,Intermittent,11/07/2024,10/03/2024,Warren W Fane,L11719,NY,Corporation,2.0
49812,3004145,New Mine,11/18/2024,NaN,NaN,NaN,NY,NaN,NaN
62561,3609813,Abandoned,05/03/2010,11/23/2009,Jonathan Schilowitz,L00045,NY,Corporation,0.0


## Write about integrating this with the rest of our codebase

Take a look at the [main Public Utility Data Liberation (PUDL) codebase](https://github.com/catalyst-cooperative/pudl).

It's certainly a little overwhelming to jump into. But we're curious how you'd approach it.

We'd like you to answer the following questions in a notebook cell below. Please show your thought process - we're particularly interested in how you deal with any *wrong* turns during your investigation. If any questions come up in this process, please note them down also.

* Where would you put this MSHA extraction code you just wrote? How do you expect the input and output to be different?

As a starting point:
* the main body of our source code lives in [`/src/pudl`](https://github.com/catalyst-cooperative/pudl/tree/main/src/pudl)
* we use [Dagster](https://docs.dagster.io/getting-started) as an ETL orchestration tool, so that might help demystify things as well.

## The MSHA extraction code would live in /src/pudl/extract. 

Looking within the src/pudl code, I first thought the place this code could live in could be either at  **analysis** , **workspace**, or **extract** because within this code, I extracted MSHA data, analyzed it, and was under the impression that it would be used/worked into. 

#### **Extract**:
I started digging around in Extract and the more I dug around, the more it made sense that it would live in extract. Files likes parquet.py and nretlab.py were doing the same things I was, extracting files and turning them into dataframes for use. But, I still thought maybe it would be in workspace since this code would need to be reviewed and worked on further. The description in Extract's init.py, "Each module in this subpackage implements data extraction for a single data source from
the PUDL :ref:`data-sources`. This process begins with the original data as retrieved by the :mod:`pudl.workspace` subpackage" made me think maybe I was missing an extra step where the code I extracted has to be first fed through to PUDL and stay in the singular data source. This made me look to pudl.workspace, I believed that workspace would have the answer to where the data would have to go/be stored.

#### **Workspace**
I looked into the init.py of the workspace and then realized that this wouldn't be the place for the MSHA extract to live in. It "downloaded data and organized it locally," it was just making the data presentable for use, not really doing the extracting that I did with the work. However, I still think this would be another place for the MSHA code to live in because it is doing the work of organizing the data and downloading it from public sources. 

#### **Analysis**
As soon as I looked into the files at Analysis, I realized this wasn't where the code should be. Yes, I extracted only NY mines but there was so much more to do and so much more left to do with the df I produced. Analysis would be more of the next step, figuring out how many companies owned mines and how many people worked in the mines. Analysis would be the next step in which I would answer the questions I brought up when selecting my columns. 

## Input and Output differences 

#### **Real input and output**
In my specific notebook, the input would have to be the file name after the user manually unzips the file. The output would be a dataset of mines specifically in New York State. 

#### **Ideal input and output**
1. Ideally, however, I would’ve liked my input to just be the zip file itself so that there wouldn’t have to be the extra step of manually opening and storing the file. (I think working with a file instead of a link here, when both are available, is better since the link/URL can be taken down at any time but the file stays in the computer/laptop) 
2. I would’ve liked my function’s parameters to be both the file and state so that the output could work for any state. For example, Mines (file, AL) would spit out mines specifically in Alabama. 
3. I would've made the function more smooth by coercing NaN values to 0 in columns that had integer value so that in the future, calculations could run more smoothly. I would've coerced NaN in noun columns (names, places) too but I think I would just leave it/follow the naming conventions of the codebase instead in order to skirt confusion.